In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 3)
    return dog_files, dog_targets

# load train, test, and validation datasets
#train_files, train_targets = load_dataset('data/train')
#test_files, test_targets = load_dataset('data/test')
#valid_files, valid_targets = load_dataset('data/valid')

Using TensorFlow backend.


(1882,)
(1882, 3)


In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [16]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
#train_tensors = paths_to_tensor(train_files).astype('float32')/255
#valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
#test_tensors = paths_to_tensor(test_files).astype('float32')/255


100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [00:41<00:00,  5.30it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [04:31<00:00,  1.99it/s]


(1882, 224, 224, 3)

In [18]:
#np.savez("data.npz", train_tensors = train_tensors, train_targets=train_targets,
#         test_tensors=test_tensors, test_targets=test_targets,
#         valid_tensors=valid_tensors, valid_targets=valid_targets)

In [3]:
data = np.load("data.npz")

train_tensors=data['train_tensors']
train_targets=data['train_targets']

test_tensors=data['test_tensors']
test_targets=data['test_targets']

valid_tensors=data['valid_tensors']
valid_targets=data['valid_targets']

print(train_tensors.shape)
print(train_targets.shape)
print(test_tensors.shape)
print(test_targets.shape)
print(valid_tensors.shape)
print(valid_targets.shape)

(1882, 224, 224, 3)
(1882, 3)
(600, 224, 224, 3)
(600, 3)
(126, 224, 224, 3)
(126, 3)


In [4]:
def extract_InceptionV3(tensor):
    from keras.applications.inception_v3 import InceptionV3, preprocess_input
    return InceptionV3(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

In [7]:
train_bn = extract_InceptionV3(train_tensors)
test_bn = extract_InceptionV3(test_tensors)
valid_bn = extract_InceptionV3(valid_tensors)

np.savez("data_inception.npz", train_bn = train_bn, train_targets=train_targets,
         test_bn=test_bn, test_targets=test_targets,
         valid_bn=valid_bn, valid_targets=valid_targets)

KeyboardInterrupt: 

(224, 224, 3)

In [8]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=train_bn.shape[1:]))

model.add(Dense(512, activation='relu', kernel_initializer='random_uniform',
                bias_initializer='random_uniform'))
model.add(Dropout(0.25))

model.add(Dense(512, activation='relu', kernel_initializer='random_uniform',
                bias_initializer='random_uniform'))
model.add(Dropout(0.25))

model.add(Dense(133, activation='softmax', kernel_initializer='random_uniform',
                bias_initializer='random_uniform'))

model.summary()

NameError: name 'train_bn' is not defined

In [9]:
from keras.optimizers import Adam, rmsprop
opt = rmsprop(lr=0.00001, decay=1e-6)
#opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

TypeError: Sequential model cannot be built: model is empty. Add some layers first.

In [10]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.inception.hdf5', 
                               verbose=1, save_best_only=True)

VGG19_model.fit(train_bn, train_targets, 
          validation_data=(valid_bn, valid_targets),
          epochs=100, batch_size=10, callbacks=[checkpointer], verbose=2)

NameError: name 'VGG19_model' is not defined

In [11]:
model.load_weights('saved_models/weights.best.inception.hdf5')

OSError: Unable to open file (Unable to open file: name = 'saved_models/weights.best.inception.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
### Calculate classification accuracy on the test dataset

predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_bn]

# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)